In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
import openai

openai.__version__

'1.13.3'

## ChatOpenAI

In [ ]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

[답변]: content='대한민국의 수도는 서울입니다.'


## PromptTemplate


In [ ]:
from langchain.prompts import PromptTemplate

# 질문 템플릿 형식 정의
template = "{country}의 수도는 뭐야?"

# 템플릿 완성
prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 뭐야?')

### LLMChain


In [ ]:
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
llm_chain.invoke({"country": "대한민국"})

{'country': '대한민국', 'text': '대한민국의 수도는 서울이야.'}

In [ ]:
llm_chain.invoke({"country": "캐나다"})

{'country': '캐나다', 'text': '캐나다의 수도는 오타와(Ottawa)입니다.'}

### apply()

`apply()` 함수로 여러개의 입력에 대한 처리를 한 번에 수행할 수 있습니다.


In [ ]:
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

response = llm_chain.apply(input_list)

In [ ]:
response[0]["text"]

'호주의 수도는 캔버라입니다.'

In [ ]:
# input_list 에 대한 결과 반환
result = llm_chain.apply(input_list)

# 반복문으로 결과 출력
for res in result:
    print(res["text"].strip())

호주의 수도는 캔버라(Canberra)입니다.
중국의 수도는 베이징(北京)입니다.
네덜란드의 수도는 암스테르담입니다.


### `generate()`


In [ ]:
# 입력값
input_list = [{"country": "호주"}, {"country": "중국"}, {"country": "네덜란드"}]

# input_list 에 대한 결과 반환
generated_result = llm_chain.generate(input_list)
print(generated_result)

generations=[[ChatGeneration(text='호주의 수도는 캔버라입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='호주의 수도는 캔버라입니다.'))], [ChatGeneration(text='중국의 수도는 베이징(北京)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='중국의 수도는 베이징(北京)입니다.'))], [ChatGeneration(text='네덜란드의 수도는 암스테르담입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='네덜란드의 수도는 암스테르담입니다.'))]] llm_output={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 58, 'total_tokens': 111}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f0b692a78'} run=[RunInfo(run_id=UUID('804b0b7e-16c3-4dd1-b1c4-c9574cb5f0f1')), RunInfo(run_id=UUID('3e1ba16a-65c5-46b9-8b3e-44ceb10e7cb6')), RunInfo(run_id=UUID('34fc2b22-227d-41f3-a8de-3e6a3ffd5eca'))]


In [ ]:
# 토큰 사용량 출력
generated_result.llm_output

{'token_usage': {'completion_tokens': 53,
  'prompt_tokens': 58,
  'total_tokens': 111},
 'model_name': 'gpt-3.5-turbo',
 'system_fingerprint': 'fp_4f0b692a78'}

### Define two or more variables in a template


In [ ]:
template = "{area1} 와 {area2} 의 시차는 몇시간이야?"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['area1', 'area2'], template='{area1} 와 {area2} 의 시차는 몇시간이야?')

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
print(llm_chain.invoke({"area1": "서울", "area2": "파리"}))

{'area1': '서울', 'area2': '파리', 'text': '서울과 파리의 시차는 8시간입니다. 서울은 GMT+9 시간대에 속하고, 파리는 GMT+1 시간대에 속하기 때문에 시차가 8시간이 발생합니다.'}


In [ ]:
input_list = [
    {"area1": "파리", "area2": "뉴욕"},
    {"area1": "서울", "area2": "하와이"},
    {"area1": "켄버라", "area2": "베이징"},
]

# 반복문으로 결과 출력
result = llm_chain.apply(input_list)
for res in result:
    print(res["text"].strip())

파리와 뉴욕의 시차는 6시간입니다. 파리는 그리니치 평균시(GMT+1)를 따르고, 뉴욕은 동부 표준시(EST)를 따르기 때문에 시차가 발생합니다.
서울과 하와이의 시차는 서울이 하와이보다 19시간 빠릅니다. 서울은 GMT+9 시간대에 있고, 하와이는 GMT-10 시간대에 있기 때문입니다.
켄버라와 베이징의 시차는 2시간입니다. 켄버라는 UTC+10 시간대에 속하고, 베이징은 UTC+8 시간대에 속하기 때문입니다.


## stream


In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [ ]:
# 질의내용
question = "대한민국에 대해서 300자 내외로 최대한 상세히 알려줘"

# 스트리밍으로 답변 출력
response = llm.invoke(question)

대한민국은 동아시아에 위치한 나라로, 수도는 서울이며 인구는 약 5천만 명이 넘는 대국가이다. 대한민국은 민주공화국으로 정부는 대통령 중심의 체제를 가지고 있으며, 국회는 한국 국회로 불린다. 대한민국은 한반도의 남쪽에 위치하고 있으며 북쪽으로는 북한과 접하고 있다.

대한민국은 세계에서 11번째로 경제 규모가 큰 나라로, 선진국가로 분류되며 고도의 경제력을 보유하고 있다. 주요 산업은 반도체, 자동차, 조선, 철강, 화학 등이 있으며, 세계적으로 유명한 기업들이 많이 발달해 있다.

문화적으로는 한국의 전통문화와 현대문화가 조화롭게 공존하고 있으며, 한류 열풍으로 한국의 음악, 드라마, 영화 등이 전 세계적으로 인기를 끌고 있다. 또한 한국의 전통 요리인 김치, 불고기, 비빔밥 등도 많은 사람들에게 사랑받고 있다.

대한민국은 안전하고 발전된 나라로, 교육 수준이 높고 의료 시스템도 잘 발달해 있어 건강하고 안정적인 삶을 누릴 수 있는 나라이다. 또한 자연환경도 아름다워 해변, 산, 계곡 등 다양한 관광지가 있어 많은 관광객들이 방문하고 있다.